# 2.5 Galaxy Tab S6 크롤링

## 2.5.1 크롤링할 페이지 접속

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [56]:
url = 'https://m.bunjang.co.kr/'

driver = webdriver.Chrome()

driver.get(url)
print(f'현재 URL : {driver.current_url}')
print(f'웹페이지 제목 : {driver.title}')

현재 URL : https://m.bunjang.co.kr/
웹페이지 제목 : 번개장터


In [57]:
# 검색창의 위치를 XPath를 이용해 변수에 저장
search_box = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[2]/div[1]/div[1]/div[1]/div[1]/input')

# 입력시에 핵심 키워드를 ' '로 구분하여 입력. 자세한 설명은 2.6.3에서 참고
item_name = input('검색하고자 하는 상품명을 입력하세요: ')

검색하고자 하는 상품명을 입력하세요: 갤럭시 탭 s6


In [58]:
# search_box 초기화를 위해 search_box에 있는 text 전체 선택
search_box.click();
search_box.send_keys(Keys.CONTROL + 'A');

# 입력한 상품 검색
search_box.send_keys(item_name)
search_box.send_keys(Keys.RETURN)

In [59]:
# 카테고리 리스트의 위치를 변수에 저장
categories = driver.find_elements(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div[1]/div/div[2]/a/div[2]')

# 카테고리들의 번호와 내용 출력
for num, c in enumerate(categories):
    print(f'{num+1}. {c.text}')

category_selected = int(input('검색하고자 하는 카테고리의 번호를 입력하세요: '))

1. 태블릿
2. 케이스/보호필름/액세서리
3. 스마트폰
4. 케이블/충전기/주변기기
검색하고자 하는 카테고리의 번호를 입력하세요: 1


In [60]:
# 선택한 카테고리의 XPath를 변수에 저장
category_path = '//*[@id="root"]/div/div/div[4]/div/div[1]/div/div[2]/a'+'['+str(category_selected)+']'

# 선택한 카테고리 클릭
search_button = driver.find_element(By.XPATH, category_path)
search_button.click()

In [61]:
# 최신순 클릭
newest = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div[3]/div/div[2]/a[2]')
newest.click()

## 2.5.2 상품 데이터 크롤링

In [67]:
import pandas as pd

# 크롤링한 데이터를 저장할 DataFrame
df = pd.DataFrame(columns=['pid', 'title', 'price', 'upload_time', 'place', 'inspection', 'delivery_fee', 'pay', 'href'])

# 크롤링 종료 조건을 확인하기 위한 flag 변수
crawling_fin_flag = 0

In [68]:
# 검색 결과 페이지 수만큼 반복 (최대 10페이지까지)
for i in range(10):
    
    product_list = driver.find_elements(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div[4]/div/div')
    
    
    ##### 페이지 별 갯수 확인 코드
    start_len = len(df)
    ############################
    
    for data in product_list:
        
        '''크롤링 종료 조건'''
        # '판매완료'인 상품 체크
        try:
            if data.find_element(By.XPATH, 'a/div[1]/div[2]/div[1]/img').get_attribute('alt') == '판매 완료':
                crawling_fin_flag = 1
                break
        except:
            pass
        
        
        '''크롤링 예외 조건'''
        # 예외조건1 - 예약중
        try:
            if data.find_element(By.XPATH, 'a/div[1]/div[2]/div[1]/img').get_attribute('alt') == '예약중':
                continue
        except:
            pass
        
        # 예외조건2 - AD 상품
        text_list = data.text.split('\n')
        
        if text_list[-2] == 'AD':
            continue


        '''DataFrame에 저장'''
        # data-pid
        pid = data.find_element(By.XPATH, 'a').get_attribute('data-pid')

        # href
        href = data.find_element(By.XPATH, 'a').get_attribute('href')
        
        
        # 번개페이 가능 여부
        try:
            result = data.find_element(By.CLASS_NAME, 'styled__IconBadge-sc-3zkh6z-2.iDmRbz')
            pay = 1
        except:
            pay = 0
        
        
        # 게시글 제목, 상품 가격, 게시글 업로드 시간, 거래 장소
        title = text_list[-4]
        price = text_list[-3]
        upload_time = text_list[-2]
        place = text_list[-1]
        
        # 검수 가능 여부
        if '검수가능' in text_list:
            inspection = 1
        else:
            inspection = 0

        # 배송비 포함 여부
        if '배송비포함' in text_list:
            delivery_fee = 1
        else:
            delivery_fee = 0


        # 데이터프레임에 저장
        df.loc[len(df)] = [pid, title, price, upload_time, place, inspection, delivery_fee, pay, href]
            
    ##### 페이지 별 갯수 확인 코드
    print(f'{i+1} page {len(df) - start_len}개 (총 {len(df)}개)')
    ############################
    
    
    # 크롤링 종료 조건
    if crawling_fin_flag:
        print('Crawling Finish')
        break
    
    # 다음 페이지 버튼 클릭
    page_button = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div[5]/div/a['+str(i+3)+']')
    page_button.click()
    
    time.sleep(2)

1 page 96개 (총 96개)
2 page 78개 (총 174개)
3 page 67개 (총 241개)
4 page 65개 (총 306개)
5 page 11개 (총 317개)
Crawling Finish


In [69]:
# webdriver 종료
driver.close()

In [70]:
# 크롤링한 데이터 확인
df

,pid,title,price,upload_time,place,inspection,delivery_fee,pay,href
0,229402912,갤럭시탭 s6 lite 64gb wifi,"190,000",1분 전,전국,0,1,1,https://m.bunjang.co.kr/products/229402912?q=%...
1,229135328,갤럭시탭s6lite 64 wife,"250,000",1시간 전,경기도 안산시 상록구 반월동,0,1,1,https://m.bunjang.co.kr/products/229135328?q=%...
2,229428258,갤럭시탭 s6 라이트,"500,000",2시간 전,경기도 남양주시 오남읍,0,1,1,https://m.bunjang.co.kr/products/229428258?q=%...
3,229329594,갤럭시탭S6 lite SM-P610 wifi 64G 판매해요.,"230,000",1시간 전,전국,0,0,1,https://m.bunjang.co.kr/products/229329594?q=%...
4,227587188,갤럭시탭s6 lite 64g 셀룰러 팝니다,"230,000",1시간 전,전국,0,1,1,https://m.bunjang.co.kr/products/227587188?q=%...
...,...,...,...,...,...,...,...,...,...
312,226465498,탭S6라이트 그레이 64기가 와이파이 (LGU+ TV프리3),"310,000",1달 전,경상북도 경주시 황오동,0,1,1,https://m.bunjang.co.kr/products/226465498?q=%...
313,226157857,탭S6라이트 그레이 64기가 와이파이 (LGU+ TV프리3),"310,000",1달 전,경상북도 경주시 황오동,0,1,1,https://m.bunjang.co.kr/products/226157857?q=%...
314,223192438,탭S6라이트 그레이 64기가 와이파이 (LGU+ TV프리3),"355,000",2달 전,경상북도 경주시 황오동,0,1,1,https://m.bunjang.co.kr/products/223192438?q=%...
315,219594988,겔탭s6lite,"380,000",3달 전,대구광역시 달서구 본리동,0,0,1,https://m.bunjang.co.kr/products/219594988?q=%...


** *** **

# 2.6 데이터 전처리

In [169]:
df_save = df.copy()

In [ ]:
# 전처리 과정 중에 데이터 원본이 다시 필요한 경우 아래 코드 실행
df = df_save.copy()

## 2.6.1 중복 상품 제거

In [170]:
# 중복행 확인
df[df.duplicated(subset=['pid'], keep='first')]

,pid,title,price,upload_time,place,inspection,delivery_fee,pay,href
174,227292927,갤럭시탭s6 LTE 라이트아닙니다 상태극상 급처,"310,000",3주 전,전국,0,0,0,https://m.bunjang.co.kr/products/227292927?q=%...


In [171]:
# 중복행 제거
df = df.drop_duplicates(subset=['pid'],keep='first')

## 2.6.2 상품 속성값 변경

#### price column 전처리

In [172]:
for index, row in df.iterrows():
    price_str = row.copy()['price']  # SettingWithCopyWarning 발생 방지를 위한 copy
    if price_str == '연락요망':  # 가격이 표시되지 않은 제품 제외
        df = df.drop(index)
    else:
        price_int = int(price_str.replace(',', ''))
        df.loc[index, 'price'] = price_int

In [173]:
df['price'].unique()

array([190000, 250000, 500000, 230000, 300000, 233000, 263000, 320000,
       390000, 150000, 220000, 450000, 313000, 355000, 253000, 285000,
       290000, 430000, 400000, 280000, 240000, 340000, 350000, 275000,
       260000, 330000, 225000, 200000, 605000, 160000, 267000, 270000,
       380000, 449000, 360000, 388000, 1000000, 345000, 457000, 420000,
       288000, 210000, 480000, 414000, 337000, 600000, 30000, 153000,
       375000, 278000, 327000, 325300, 367000, 750000, 310000, 539000,
       1725000, 100000000, 519000, 354000, 800000, 370000, 100000, 100,
       170000, 410000, 10000, 22222, 550000, 552000, 1000, 374000, 440000],
      dtype=object)

#### upload time column 전처리

In [174]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
now = datetime.now()

for index, row in df.iterrows():
    if df.loc[index, 'upload_time'][-3:] == '초 전':
        df.loc[index, 'upload_time'] = (now - timedelta(seconds=int(df.loc[index, 'upload_time'][:-3]))).strftime('%Y-%m-%d')
    elif df.loc[index, 'upload_time'][-3:] == '분 전':
        df.loc[index, 'upload_time'] = (now - timedelta(minutes=int(df.loc[index, 'upload_time'][:-3]))).strftime('%Y-%m-%d')
    elif df.loc[index, 'upload_time'][-4:] == '시간 전':
        df.loc[index, 'upload_time'] = (now - timedelta(hours=int(df.loc[index, 'upload_time'][:-4]))).strftime('%Y-%m-%d')
    elif df.loc[index, 'upload_time'][-3:] == '일 전':
        df.loc[index, 'upload_time'] = (now - timedelta(days=int(df.loc[index, 'upload_time'][:-3]))).strftime('%Y-%m-%d')
    elif df.loc[index, 'upload_time'][-3:] == '주 전':
        df.loc[index, 'upload_time'] = (now - timedelta(weeks=int(df.loc[index, 'upload_time'][:-3]))).strftime('%Y-%m-%d')
    elif df.loc[index, 'upload_time'][-3:] == '달 전':
        df.loc[index, 'upload_time'] = (now - relativedelta(months=int(df.loc[index, 'upload_time'][:-3]))).strftime('%Y-%m-%d')
    else:
        print(df.loc[index, 'upload_time'])
        df.loc[index, 'upload_time'] = (now - relativedelta(years=int(df.loc[index, 'upload_time'][:-3]))).strftime('%Y-%m-%d')

In [175]:
df['upload_time'].unique()

array(['2023-07-09', '2023-07-08', '2023-07-07', '2023-07-06',
       '2023-07-05', '2023-07-04', '2023-07-03', '2023-07-02',
       '2023-06-25', '2023-06-18', '2023-06-11', '2023-06-09',
       '2023-05-09', '2023-04-09', '2023-03-09', '2023-02-09',
       '2023-01-09', '2022-12-09', '2022-11-09', '2022-10-09',
       '2022-09-09', '2022-08-09', '2022-07-09'], dtype=object)

## 2.6.3 불필요한 상품 제거

#### title column 전처리

In [176]:
# '구매' 목적인 게시글의 상품 제외
import re
pattern = r'삽니다|구해|구합|구매|사요'
pattern_matching_index = df[df['title'].str.contains(pattern, flags=re.IGNORECASE, regex=True)].index

df = df.drop(index=pattern_matching_index)

In [177]:
# 검색어와 관련없는 상품 제외
item_name_list = item_name.split(' ')  # 검색어 입력시 키워드를 공백으로 구분

for index, row in df.iterrows():
    title = row['title'].replace(' ','').lower()  # 알파벳은 대소문자 구분하지 않음
    
    for check in item_name_list[1:]:  # '갤럭시', '겔럭시', 'Galaxy'가 많아서 '탭'과 's6'로만 확인
        if check.lower() not in title:
            df = df.drop(index)
            break

In [178]:
df

,pid,title,price,upload_time,place,inspection,delivery_fee,pay,href
0,229402912,갤럭시탭 s6 lite 64gb wifi,190000,2023-07-09,전국,0,1,1,https://m.bunjang.co.kr/products/229402912?q=%...
1,229135328,갤럭시탭s6lite 64 wife,250000,2023-07-09,경기도 안산시 상록구 반월동,0,1,1,https://m.bunjang.co.kr/products/229135328?q=%...
2,229428258,갤럭시탭 s6 라이트,500000,2023-07-09,경기도 남양주시 오남읍,0,1,1,https://m.bunjang.co.kr/products/229428258?q=%...
3,229329594,갤럭시탭S6 lite SM-P610 wifi 64G 판매해요.,230000,2023-07-09,전국,0,0,1,https://m.bunjang.co.kr/products/229329594?q=%...
4,227587188,갤럭시탭s6 lite 64g 셀룰러 팝니다,230000,2023-07-09,전국,0,1,1,https://m.bunjang.co.kr/products/227587188?q=%...
...,...,...,...,...,...,...,...,...,...
312,226465498,탭S6라이트 그레이 64기가 와이파이 (LGU+ TV프리3),310000,2023-06-09,경상북도 경주시 황오동,0,1,1,https://m.bunjang.co.kr/products/226465498?q=%...
313,226157857,탭S6라이트 그레이 64기가 와이파이 (LGU+ TV프리3),310000,2023-06-09,경상북도 경주시 황오동,0,1,1,https://m.bunjang.co.kr/products/226157857?q=%...
314,223192438,탭S6라이트 그레이 64기가 와이파이 (LGU+ TV프리3),355000,2023-05-09,경상북도 경주시 황오동,0,1,1,https://m.bunjang.co.kr/products/223192438?q=%...
315,219594988,겔탭s6lite,380000,2023-04-09,대구광역시 달서구 본리동,0,0,1,https://m.bunjang.co.kr/products/219594988?q=%...


## 2.6.4 상품 속성 Column 추가 생성

#### volume column 생성 및 값 저장

In [179]:
df['volume'] = ''

for index, row in df.iterrows():
    title = row['title'].replace(' ','').lower()
    if '64' in title:
        df.loc[index, 'volume'] = '64GB'
    elif '128' in title:
        df.loc[index, 'volume'] = '128GB'
    elif '256' in title:
        df.loc[index, 'volume'] = '256GB'
    else:
        df.loc[index, 'volume'] = '-'

#### version column 생성 및 값 저장

In [180]:
df['version'] = ''
    
flag_dict = {'5g':0, 'wifi': 0, 'lte': 0}
for index, row in df.iterrows():
    title = row['title'].replace(' ','').lower()

    # '5g', 'wifi', 'lte' 기종인지 확인
    if '5g' in title or '866' in title:
        flag_dict['5g'] = 1
    if ('wifi' in title or '와이파이' in title or '610' in title or '860' in title):
        flag_dict['wifi'] = 1
    if ('lte' in title or '룰러' in title or '615' in title or '865' in title):
        flag_dict['lte'] = 1
    
    # 확인한 데이터를 토대로 '5G', 'WIFI', 'LTE' 저장
    if flag_dict['5g'] == 1:
        df.loc[index, 'version'] = '5G'
        df.loc[index, 'volume'] = '128GB'  # 5G 기종의 용량은 모두 128G
    elif flag_dict['wifi'] == 1 and flag_dict['lte'] == 1:
        df.loc[index, 'version'] = 'WIFI/LTE'
    elif flag_dict['wifi'] == 1:
        df.loc[index, 'version'] = 'WIFI'
    elif flag_dict['lte'] == 1:
        df.loc[index, 'version'] = 'lte'
    else:
        df.loc[index, 'version'] = '-'
    
    # flag 값 초기화
    flag_dict['5g'] = 0
    flag_dict['wifi'] = 0
    flag_dict['lte'] = 0

#### type colmun 생성 및 값 저장

In [181]:
df['type'] = ''

for index, row in df.iterrows():
    if row['volume'] == '64GB':
        df.loc[index, 'type'] = 'S6 Lite'
    elif row['volume'] == '256GB':
        df.loc[index, 'type'] = 'S6'
    elif '61' in row['title']:
        df.loc[index, 'type'] = 'S6 Lite'
    elif '86' in row['title']:
        df.loc[index, 'type'] = 'S6'
    else:  
        title = row['title'].replace(' ','').lower()
        if len(re.findall(r'(lite|라이트)(아님|아닙|X)', title)) > 0:
            df.loc[index, 'type'] = 'S6'
        elif 'lite' in title or '라이트' in title:
            df.loc[index, 'type'] = 'S6 Lite'
        else:
            df.loc[index, 'type'] = 'S6'

In [182]:
df.head()

,pid,title,price,upload_time,place,inspection,delivery_fee,pay,href,volume,version,type
0,229402912,갤럭시탭 s6 lite 64gb wifi,190000,2023-07-09,전국,0,1,1,https://m.bunjang.co.kr/products/229402912?q=%...,64GB,WIFI,S6 Lite
1,229135328,갤럭시탭s6lite 64 wife,250000,2023-07-09,경기도 안산시 상록구 반월동,0,1,1,https://m.bunjang.co.kr/products/229135328?q=%...,64GB,-,S6 Lite
2,229428258,갤럭시탭 s6 라이트,500000,2023-07-09,경기도 남양주시 오남읍,0,1,1,https://m.bunjang.co.kr/products/229428258?q=%...,-,-,S6 Lite
3,229329594,갤럭시탭S6 lite SM-P610 wifi 64G 판매해요.,230000,2023-07-09,전국,0,0,1,https://m.bunjang.co.kr/products/229329594?q=%...,64GB,WIFI,S6 Lite
4,227587188,갤럭시탭s6 lite 64g 셀룰러 팝니다,230000,2023-07-09,전국,0,1,1,https://m.bunjang.co.kr/products/227587188?q=%...,64GB,lte,S6 Lite


***

# 2.7 MySQL DB에 데이터 적재

## 2.7.1 DB 생성

In [184]:
import pymysql

# password 입력
password = '0000'

In [185]:
# MySQL Connection
conn = pymysql.connect(host = '127.0.0.1',
                       port = 3306,
                       user = 'root',
                       password = password,
                       charset='utf8')

In [186]:
# SQL문 실행 - DB 생성
with conn:
    with conn.cursor() as cur:
        cur.execute('CREATE DATABASE lightning_Market')
        conn.commit()

## 2.7.2 테이블 생성

In [187]:
# MySQL DB Connection
conn = pymysql.connect(host = '127.0.0.1',
                       port = 3306,
                       user = 'root',
                       password = password,
                       charset='utf8',
                       database='lightning_Market')

In [188]:
# 테이블 생성 SQL문
sql = '''create table market_product (
            pid varchar(10) primary key,
            title varchar(50),
            price int,
            upload_time varchar(10),    
            place varchar(30),
            inspection int,
            delivery_fee int,
            pay int,
            href varchar(150),
            volume varchar(10),
            type varchar(10),
            version varchar(10));'''

In [189]:
# SQL문 실행 - 테이블 생성
with conn.cursor() as cur:
    cur.execute(sql)
    conn.commit()

## 2.7.3 데이터 저장

In [190]:
# SQL문 실행 - 데이터 삽입
with conn.cursor() as cursor:
    for _, row in df.iterrows():
        sql = '''
        INSERT INTO market_product (pid, title, price, upload_time, place, inspection, delivery_fee, pay, href, volume, type, version)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        '''
        
        values = (
            row['pid'], row['title'], row['price'], row['upload_time'], row['place'], 
            row['inspection'], row['delivery_fee'], row['pay'], row['href'], row['volume'], row['type'], row['version']
        )
        
        cursor.execute(sql, values)
        
    conn.commit()

In [191]:
# MySQL DB Connection 종료
conn.close()

***

# 2.8 DB 활용 - 필터링

In [206]:
# MySQL DB Connection
conn = pymysql.connect(host = '127.0.0.1',
                       port = 3306,
                       user = 'root',
                       password = password,
                       charset='utf8',
                       database='lightning_Market')

## 2.8.1 필터링1 - 용량, 지원버전, 가격, 등록 날짜 조건

In [207]:
# 필터링1 조건에 만족하는 데이터를 조회하는 SQL문
query = '''SELECT *
            FROM market_product 
            WHERE volume = '128GB'
                AND version != 'LTE'
                AND price < 350000
                AND upload_time > '2023-06-01'
            ORDER BY upload_time DESC;'''

In [208]:
# SQL문 실행 - 데이터 조회
with conn.cursor() as cursor:
    cursor.execute(query)
    result = cursor.fetchall()

In [209]:
# 결과 출력
print(f'조건에 일치하는 상품: 총 {len(result)}개')

result_df = pd.DataFrame(result, columns=['pid', 'title', 'price', 'upload_time', 'place', 'inspection', 'delivery_fee', 'pay', 'href', 'volume', 'type', 'version'])
result_df

조건에 일치하는 상품: 총 10개


,pid,title,price,upload_time,place,inspection,delivery_fee,pay,href,volume,type,version
0,226496891,갤럭시탭S6 lite wifi 128GB (sm-p610),300000,2023-07-09,전국,0,1,0,https://m.bunjang.co.kr/products/226496891?q=%...,128GB,S6 Lite,WIFI
1,229414483,갤럭시탭s6 128 wifi팝니다,320000,2023-07-09,경기도 수원시 팔달구 우만2동,0,0,1,https://m.bunjang.co.kr/products/229414483?q=%...,128GB,S6,WIFI
2,216456152,삼성 갤럭시탭 S6 LITE 128GB WIFI모델 SM-P610,340000,2023-07-07,경기도 김포시 고촌읍,0,0,1,https://m.bunjang.co.kr/products/216456152?q=%...,128GB,S6 Lite,WIFI
3,225600476,갤럭시탭 S6 Lite 128g,290000,2023-07-07,전국,0,0,1,https://m.bunjang.co.kr/products/225600476?q=%...,128GB,S6 Lite,-
4,227381535,갤럭시탭 S6 LITE S6라이트 128G 보내요,300000,2023-07-05,전국,0,1,1,https://m.bunjang.co.kr/products/227381535?q=%...,128GB,S6 Lite,-
5,228995067,갤럭시탭 S6 LITE 128기가 판매합니다,345000,2023-07-04,경기도 남양주시 진접읍,0,0,0,https://m.bunjang.co.kr/products/228995067?q=%...,128GB,S6 Lite,-
6,227222959,갤럭시탭s6 lite 128gb wifi,270000,2023-07-02,전국,0,0,1,https://m.bunjang.co.kr/products/227222959?q=%...,128GB,S6 Lite,WIFI
7,227916112,삼성 갤럭시탭 S6라이트 10.4 128G 그레이 S펜 태블릿 P610,325300,2023-06-25,서울특별시 강동구 길동,0,0,1,https://m.bunjang.co.kr/products/227916112?q=%...,128GB,S6 Lite,WIFI
8,225108202,갤럭시탭 s6lite 급처 128GB,330000,2023-06-09,전국,0,0,1,https://m.bunjang.co.kr/products/225108202?q=%...,128GB,S6 Lite,-
9,226149966,갤럭시탭 s6lite 급처 128GB,330000,2023-06-09,전국,0,0,1,https://m.bunjang.co.kr/products/226149966?q=%...,128GB,S6 Lite,-


In [210]:
result_df[result_df['pid'] == '226496891']['href']

0    https://m.bunjang.co.kr/products/226496891?q=%...
Name: href, dtype: object

## 2.8.2 필터링2 - 직거래 가능 상품

In [211]:
# 필터링2 조건에 만족하는 데이터를 조회하는 SQL문
query = '''SELECT *
            FROM market_product
            WHERE volume = '128GB'
                AND version != 'LTE'
                AND price < 350000
                AND upload_time > '2023-06-01'
                AND place LIKE '서울%'
            ORDER BY upload_time DESC;'''

In [212]:
# SQL문 실행 - 데이터 조회
with conn.cursor() as cursor:
    cursor.execute(query)
    result = cursor.fetchall()

In [213]:
# 결과 출력
print(f'조건에 일치하는 상품: 총 {len(result)}개')

result_df = pd.DataFrame(result, columns=['pid', 'title', 'price', 'upload_time', 'place', 'inspection', 'delivery_fee', 'pay', 'href', 'volume', 'type', 'version'])
result_df

조건에 일치하는 상품: 총 1개


,pid,title,price,upload_time,place,inspection,delivery_fee,pay,href,volume,type,version
0,227916112,삼성 갤럭시탭 S6라이트 10.4 128G 그레이 S펜 태블릿 P610,325300,2023-06-25,서울특별시 강동구 길동,0,0,1,https://m.bunjang.co.kr/products/227916112?q=%...,128GB,S6 Lite,WIFI


In [214]:
result_df[result_df['pid'] == '227916112']['href']

0    https://m.bunjang.co.kr/products/227916112?q=%...
Name: href, dtype: object

## 2.8.3 필터링3 - 택배 거래

In [215]:
# 필터링3 조건에 만족하는 데이터를 조회하는 SQL문
query = '''SELECT *
            FROM market_product
            WHERE volume = '128GB'
                AND version != 'LTE'
                AND price < 350000
                AND upload_time > '2023-06-01'
                AND pay = 1
            ORDER BY upload_time DESC;'''

In [216]:
# SQL문 실행 - 데이터 조회
with conn.cursor() as cursor:
    cursor.execute(query)
    result = cursor.fetchall()

In [217]:
# 결과 출력
print(f'조건에 일치하는 상품: 총 {len(result)}개')
result_df = pd.DataFrame(result, columns=['pid', 'title', 'price', 'upload_time', 'place', 'inspection', 'delivery_fee', 'pay', 'href', 'volume', 'type', 'version'])
result_df

조건에 일치하는 상품: 총 8개


,pid,title,price,upload_time,place,inspection,delivery_fee,pay,href,volume,type,version
0,229414483,갤럭시탭s6 128 wifi팝니다,320000,2023-07-09,경기도 수원시 팔달구 우만2동,0,0,1,https://m.bunjang.co.kr/products/229414483?q=%...,128GB,S6,WIFI
1,216456152,삼성 갤럭시탭 S6 LITE 128GB WIFI모델 SM-P610,340000,2023-07-07,경기도 김포시 고촌읍,0,0,1,https://m.bunjang.co.kr/products/216456152?q=%...,128GB,S6 Lite,WIFI
2,225600476,갤럭시탭 S6 Lite 128g,290000,2023-07-07,전국,0,0,1,https://m.bunjang.co.kr/products/225600476?q=%...,128GB,S6 Lite,-
3,227381535,갤럭시탭 S6 LITE S6라이트 128G 보내요,300000,2023-07-05,전국,0,1,1,https://m.bunjang.co.kr/products/227381535?q=%...,128GB,S6 Lite,-
4,227222959,갤럭시탭s6 lite 128gb wifi,270000,2023-07-02,전국,0,0,1,https://m.bunjang.co.kr/products/227222959?q=%...,128GB,S6 Lite,WIFI
5,227916112,삼성 갤럭시탭 S6라이트 10.4 128G 그레이 S펜 태블릿 P610,325300,2023-06-25,서울특별시 강동구 길동,0,0,1,https://m.bunjang.co.kr/products/227916112?q=%...,128GB,S6 Lite,WIFI
6,225108202,갤럭시탭 s6lite 급처 128GB,330000,2023-06-09,전국,0,0,1,https://m.bunjang.co.kr/products/225108202?q=%...,128GB,S6 Lite,-
7,226149966,갤럭시탭 s6lite 급처 128GB,330000,2023-06-09,전국,0,0,1,https://m.bunjang.co.kr/products/226149966?q=%...,128GB,S6 Lite,-


In [218]:
result_df[result_df['pid'] == '229414483']['href']

0    https://m.bunjang.co.kr/products/229414483?q=%...
Name: href, dtype: object

In [219]:
# MySQL DB Connection 종료
conn.close()